# Part 0 Data Preparation

In [1]:
grid = [list(line) for line in open("23.txt").read().splitlines()]

startpos = (1,0)

gw = len(grid[0])
gh = len(grid)

forests = set()
for y in range(gh):
    for x in range(gw):
        if grid[y][x] == "#":
            forests.add((x,y))

def  print_grid_old(grid):
    for line in grid:
        print("".join(line))

def print_grid(grid, positions):
    g = grid.copy()
    for y in range(gh):
        for x in range(gw):
            if(x,y) in positions:
                print("O", end="")
                
            elif(x,y) in forests:
                print("#", end="")
            else:
                print(' ', end="")
        print()

# Part 1

In [2]:
from IPython import display
paths = [(set([(1,0), startpos]), startpos)]
# print(paths)
def test(testPos, steps):
    # print("testing ", testPos)
    if testPos[0] < 0 or testPos[0] >= gw:
        # print("Out of bounds", end="")
        return False
    if testPos[1] < 0 or testPos[1] >= gh:
        # print("Out of bounds", end="")
        return False
    if testPos in steps:
        # print("Retracing steps", end ="")
        return False
    if testPos in forests:
        # print("Hitting wall", end="")
        return False
    return True
longestPath = 0
counter =0 
targetPos = ( (gw - 2), (gh - 1))
while len(paths) > 0:
    newpaths = []
    counter = counter + 1
    for path in paths:
        steps, current = path
        token = grid[current[1]][current[0]]
        if current == targetPos:
            # This is the end
            l = len(steps)
            if l > longestPath:
                longestPath = l 
        elif token == ">":
            testPos = (current[0]+1, current[1])
            if test(testPos, steps):
                newpaths.append((steps.union([testPos]), testPos))
        elif token == "<":
            testPos = (current[0]-1, current[1])
            if test(testPos, steps):
                newpaths.append((steps.union([testPos]), testPos))
        elif token == "^":
            testPos = (current[0], current[1]-1)
            if test(testPos, steps):
                newpaths.append((steps.union([testPos]), testPos))
        elif token == "v":
            testPos = (current[0], current[1]+1)
            if test(testPos, steps):
                newpaths.append((steps.union([testPos]), testPos))
        else:
            # print("great other")
            for x in [-1, 1]:
                testPos = (current[0]+x, current[1])
                if test(testPos, steps):
                    newpaths.append((steps.union([testPos]), testPos))
            for y in [-1, 1]:
                testPos = (current[0], current[1]+y)
                if test(testPos, steps):
                    newpaths.append((steps.union([testPos]), testPos))
    outpaths = []
    upaths = []
    for path in newpaths:
        steps, current = path
        if current not in [x[1] for x in upaths]:
            upaths.append(path)
    paths = upaths
print(longestPath - 1)

2214


# Part 2

*Thesis*: The maze is a bunch of corridors that basically connect a more limited number of junctions. We find those and look for the lengths between them. 

In [3]:
junctions = set()
for y in range(gh):
    if(y == 0 or y == gh-1):
        continue
    for x in range(gw):
        if(x == 0 or x == gw-1):
            continue
        # look at cardinals
        if(grid[y][x] == "#"):
            continue
        counter = 0
        if(grid[y-1][x] != "#"):
            counter += 1
        if(grid[y+1][x] != "#"):
            counter += 1
        if(grid[y][x-1] != "#"):
            counter += 1
        if(grid[y][x+1] != "#"):
            counter += 1
        if counter >= 3:
            junctions.add((x,y))
print(junctions)
print(len(junctions))

{(113, 29), (65, 101), (7, 35), (79, 109), (125, 137), (29, 105), (33, 133), (57, 11), (61, 39), (135, 101), (89, 61), (43, 39), (59, 127), (111, 15), (29, 89), (77, 135), (7, 101), (41, 5), (127, 81), (83, 17), (61, 81), (9, 77), (13, 65), (75, 83), (7, 9), (59, 53), (31, 67), (107, 65), (131, 65), (135, 29), (109, 99), (107, 135), (105, 83), (83, 37)}
34


## Get Lengths between junctions

In [4]:
# New thesis - this is made of pathways between junctions. We should look from junction to junction to find which ones connect, and of course which ones connect to the beginning and the end
junctions.add(startpos)
targetPos = ( (gw - 2), (gh - 1))
junctions.add(targetPos)
print("number of junctions", len(junctions))
# junctions
links= []
for j in junctions:
    paths = [(set([j]), j)]
    while len(paths) > 0:
        newpaths = []
        for path in paths:
            steps, current = path
            if current in junctions and len(steps) > 1: # current != j:
                l = len(steps)
                # This is the end
                links.append((j, current,l - 1, steps))
            else:
                for x in [-1, 1]:
                    testPos = (current[0]+x, current[1])
                    if test(testPos, steps):
                        newpaths.append((steps.union([testPos]), testPos))
                for y in [-1, 1]:
                    testPos = (current[0], current[1]+y)
                    if test(testPos, steps):
                        newpaths.append((steps.union([testPos]), testPos))
        paths = newpaths

linkobj = {}
linkstepsobj = {}
for link in links:
    start, end, length, steps = link
    if start not in linkobj:
        linkobj[start] = {}
    linkobj[start][end] = length
    if start not in linkstepsobj:
        linkstepsobj[start] = {}
    linkstepsobj[start][end] = steps

print("paths from target", linkobj[targetPos])

paths = [[startpos]]
finalPaths = []
counter = 0
while(len(paths) > 0):
    newpaths = []
    for path in paths:
        current = path[-1]
        if current == targetPos:
            finalPaths.append(path)
        else:
            for next in linkobj[current]:
                if next not in path:
                    newpaths.append(path + [next])
    print("turn", counter, " - number of paths to examine", len(newpaths))
    counter = counter + 1
    if(counter > 100): break #Safety valve to prevent infinite run
    paths = newpaths

number of junctions 36
paths from target {(125, 137): 57}
turn 0  - number of paths to examine 1
turn 1  - number of paths to examine 2
turn 2  - number of paths to examine 4
turn 3  - number of paths to examine 10
turn 4  - number of paths to examine 24
turn 5  - number of paths to examine 60
turn 6  - number of paths to examine 146
turn 7  - number of paths to examine 352
turn 8  - number of paths to examine 816
turn 9  - number of paths to examine 1812
turn 10  - number of paths to examine 3812
turn 11  - number of paths to examine 7834
turn 12  - number of paths to examine 15068
turn 13  - number of paths to examine 29306
turn 14  - number of paths to examine 53692
turn 15  - number of paths to examine 98656
turn 16  - number of paths to examine 172462
turn 17  - number of paths to examine 296710
turn 18  - number of paths to examine 488322
turn 19  - number of paths to examine 775104
turn 20  - number of paths to examine 1175472
turn 21  - number of paths to examine 1679102
turn 2

## Part 2 Final Answer

In [5]:
distances = []
stepdistances = []
longestPath = None
for f in finalPaths:
    totalDistance = 0
  
    for i in range(len(f)-1):
        start = f[i]
        end = f[i+1]
        distance = linkobj[start][end]
       
        totalDistance += distance

        
    distances.append(totalDistance)

maxDistance = max(distances)
for j, d in enumerate(distances):
    if d == maxDistance:
        break

f = finalPaths[j]

stepset = set()
for i in range(len(f)-1):
    start = f[i]
    end = f[i+1]
    steps = linkstepsobj[start][end]
    for step in steps:
        if step not in stepset:
            stepset.add(step)

# print("number of steps (Including starting position)", len(stepset))
print("Maximum Hike Distance", maxDistance)


Maximum Hike Distance 6594


## Post-answer visualization

In [6]:
print(startpos in stepset)

len(finalPaths)

print_grid(grid, stepset)

True
#O###########################################################################################################################################
#OOO#OOOOOOOOO#OOO#OOOOOOO#OOOOO###OOOOOOOOO#OOOOO#OOO#OOO#OOO###OOO#OOOOOOOOOOO#OOOOO#OOOOO#OOOOOOO#OOO###OOO#OOO###OOOOOOO###OOOOOOOOO#OOO#
###O#O#######O#O#O#O#####O#O###O###O#######O#O###O#O#O#O#O#O#O###O#O#O#########O#O###O#O###O#O#####O#O#O###O#O#O#O###O#####O###O#######O#O#O#
###O#O#OOOOOOO#O#O#OOOOO#O#OOO#O###O#OOOOOOO#OOO#OOO#OOO#O#O#O###O#O#OOOOO#OOOOO#OOO#O#O#OOO#OOO#OOO#O#O#OOO#O#O#OOO#OOOOO#OOO#O#OOOOOOO#O#O#
###O#O#O#######O#O#####O#O###O#O###O#O#########O#########O#O#O###O#O#####O#O#######O#O#O#O#####O#O###O#O#O###O#O###O#####O###O#O#O#######O#O#
#OOO#O#OOO#OOO#O#O#OOO#O#O#OOO#O#OOO#O#OOOOO###O#OOOOOOOOO#O#OOOOO#OOOOO#O#O#OOO###O#O#O#OOO###O#O###O#O#OOO#O#OOO#OOO###O#OOO#O#OOOOO#OOO#O#
#O###O###O#O#O#O#O#O#O#O#O#O###O#O###O#O# #O###O#O#########O###########O#O#O#O#O###O#O#O###O###O#O###O#O###O#O###O###O###O#O###O#####O#O###O#
#